In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import random
import numpy as np
import pandas as pd

from keras import layers

C:\Users\thsun\anaconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Number of sentences + 1 for padding.
max_features = 35001

# Inputs and embeddings.
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 768, trainable=False)(inputs)

# Bi-GRU.
x = layers.Bidirectional(layers.GRU(32, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(32, return_sequences=True))(x)

# Classifier.
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ embedding (Embedding)           │ (None, None, 768)         │ 26,880,768 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 64)          │    153,984 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ bidirectional_1 (Bidirectional) │ (None, None, 64)          │     18,816 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, None, 1)           │         65 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 27,053,633 (103.20 MB)

 Trainable params: 27,053,633 (103.20 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
embedding = np.zeros((35001, 768))
with open('task3_sentence_embedding.csv', 'r', newline='') as f:
    idx = 0
    for line in f:
        embedding[idx] = np.array([float(i) for i in line.split()])
        idx += 1

df = pd.read_json("Task 3-20231125T064001Z-001\Task 3\MELD_train_efr.json")

X = []
idx = 0
for utterances in df["utterances"]:
    x = []
    for utterance in utterances:
        x.append(idx)
        idx += 1
    X.append(x)

Y = []

for triggers in df["triggers"]:
    Y.append([0 if trigger == None else int(trigger) for trigger in triggers])

max_len = max([len(i) for i in X])
X = keras.utils.pad_sequences(X, maxlen=max_len, padding='post', value=35000)
Y = keras.utils.pad_sequences(Y, maxlen=max_len, padding='post')

In [4]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, Y, batch_size=32, epochs=5, validation_split=0.1)

Epoch 1/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 26s 226ms/step - accuracy: 0.9316 - loss: 0.2702 - val_accuracy: 0.9540 - val_loss: 0.1063
Epoch 2/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 25s 218ms/step - accuracy: 0.9672 - loss: 0.0733 - val_accuracy: 0.9531 - val_loss: 0.1092
Epoch 3/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 28s 249ms/step - accuracy: 0.9930 - loss: 0.0188 - val_accuracy: 0.9569 - val_loss: 0.1199
Epoch 4/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 25s 225ms/step - accuracy: 0.9990 - loss: 0.0038 - val_accuracy: 0.9556 - val_loss: 0.1276
Epoch 5/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 27s 236ms/step - accuracy: 0.9999 - loss: 5.5476e-04 - val_accuracy: 0.9564 - val_loss: 0.1434


In [5]:
out = model.predict(X, batch_size=32)
out = out.squeeze() > 0.5
with np.printoptions(threshold=np.inf):
    print(out)

125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step
[[False False False  True False False False False False False False False
  False False False False False False False False False False False False]
 [False False False False False  True False False False False False False
  False False False False False False False False False False False False]
 [False False False False False False False  True False  True False False
  False False False False False False False False False False False False]
 [False False False False False False False False False False False  True
  False False False False False False False False False False False False]
 [False False  True False False False False False False False False False
  False False False False False False False False False False False False]
 [False False False  True False False False False False False False False
  False False False False False False False False False False False False]
 [False False False False  True  True False False False False F